In [113]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
import numpy as np
import timeit
from math import *

a = 3
s = 3
r = 3
rr = 20
ro = 70
ra = 300
noise = 0.01
prop = 0.4
weight = 3
bias = np.array([0.0,-1.0])
dev_bias = 3
dTheta = 120
sight_theta = 180


In [163]:
def initialize_agents(speed, N, width, height):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    seed()

    agents = [np.array([0.0, 0.0]) for i in range(N)]
    #agents = [Point(uniform(0, width), uniform(0, height)) for i in range(N)]
    
    speeds = N*[0,0]
    #speeds = [np.array([0.0, 0.0]) for i in range(N)]
    
    for i in range(N):
        theta = uniform(0, 2 * np.pi)
        #speeds[i][0] = speed * np.cos(theta)
        #speeds[i][1] = speed * np.sin(theta)
        speeds[i]= speed * np.array([np.cos(theta),np.sin(theta)])
        
        theta = uniform(0, 2 * np.pi)
        radius = uniform(0,height/2)
        agents[i] = Point(width/2 + radius*np.cos(theta), height/2 + radius*np.sin(theta))

    return agents, speeds


def initialize_window(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
        weightedSpeed = a * speeds[nearest_neighbours[i]]
        speeds[i] = speeds[i] + weightedSpeed
        speeds[i] = s * normalized(speeds[i])


def angle(vec1, vec2):  # (Not so) stupid way of doing an angle
    return np.arccos(np.dot(normalized(vec1),normalized(vec2))) # radians
            
def get_distances(agent, agents, N):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    dists = N * [0.0]
    for i in range(N):
        a, dists[i] = relative_pos(agent, agents[i])
        if dists[i] == 0:
            dists[i] = 0.1
    
    return dists

def relative_pos(agent1, agent2):
    dx = agent2.getX() - agent1.getX()
    dy = agent2.getY() - agent1.getY()

    return np.array([dx, dy]), np.linalg.norm([dx, dy])

def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents, N)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def softened_angle(speed, newspeed, s, maxTheta):
    theta = angle(speed, newspeed)
    if maxTheta**2 > theta**2:
        return s*newspeed
    else:
        return np.dot(rot_matrix(maxTheta), speed)    

def in_sight_range(rel_pos, speed1, angle_range):
    return 360*angle(speed1, rel_pos)/np.pi < angle_range
    

def noisy_vector(noise):
    return noise * np.array([2 * random() - 1, 2 * random() - 1])
    
def biaser(speeds, N, s, i, prop, bias, dev_bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * prop)
    gbias = np.random.normal(bias, dev_bias, )
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * gbias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir
    #bias = np.dot(tot_dir,np.array([[np.cos(rot_bias*i), 0],[0, np.sin(rot_bias*i)]]))
    #return bias

def quality (agents, speeds, N, bias, window, old_cm):
#ACCURACY (DIRECTIONS DISTRIBUTION)
    dev = 0.0
    for i in range(N):
        dev += angle(bias, speeds[i])
    dev_avg = dev/(N*2*(np.pi))
    #print (dev_avg)
    
#ELONGATION: SHAPE OF SWARM
    #can be smarter if we make agents become poses before, more globally in  the code
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    #Center of Mass
    cm = np.mean(poses,axis=0)
    #Standard Deviation
    std = np.std(poses,axis=0)
    #print (cm, std)

    #Elongation >1 means 
    elong = std[1] / std[0]
    #Drawing it
    #old_cm = Point(cm[0],cm[1])
    #old_cm.draw(window)
    #old_cm.setFill("red")

#GROUP DIRECTION
    #Vector
    group_dir = np.array([cm[0] - old_cm[0], cm[1] - old_cm[1]])                
    #print (group_dir)
    # Norm
    group_dir = np.linalg.norm(group_dir)
    #print (group_dir)
    return cm
    


def rigid_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


def rot_matrix(theta, unit = "None"):
    if unit in ["None", "deg"]:
        c, s = np.cos(pi * theta / 180), np.sin(pi * theta / 180)
    elif unit == "rad":
        c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],[s, c]])

def warn_me_args(N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dTheta):
    if dt*s < rr:
        print("Warning - step length bigger the repulsion radius.")
        

        
def virtualizer (current, agents, h, w, N):
    lower_limit = current.getY() - h / 2
    upper_limit = current.getY() + h / 2
    left_limit = current.getX() - w / 2
    right_limit = current.getX() + w / 2
    
    vagents=[np.array([agent.getX(),agent.getY()]) for agent in agents]
    virtuals = N * [0.0, 0.0]
    vvirtuals = N * [Point(0.0, 0.0)]

    for j in range(N):
        #make more compact!!!
        virtuals[j]=vagents[j]
        
        #newcopy = agents[i].clone()
        candidates = [vagents[j],
                      vagents[j]+[w,0],vagents[j]+[-w,0],vagents[j]+[w,h],vagents[j]+[w,-h],
                      vagents[j]+[-w,h],vagents[j]+[-w,-h],vagents[j]+[0,h],vagents[j]+[0,-h]]
        #print candidates
        
        #virtuals[j] = next((cand for cand in candidates if lower_limit < cand[1] < upper_limit and left_limit < cand[0] < right_limit),False
        for i in range(9):
            if lower_limit < candidates[i][1] < upper_limit and left_limit < candidates[i][0] < right_limit:
                virtuals[j]=candidates[i]
            
        vvirtuals[j] = Point(virtuals[j][0],virtuals[j][1])
        
    return vvirtuals


def get_cm_std(agents, N):
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    return np.mean(poses,axis = 0), np.std(poses,axis = 0)

def mill_observables (N, agents, speeds):
    cm, std = get_cm_std(agents, N)
    mean_R = np.linalg.norm(std)
    point_cm = Point(cm[0],cm[1])
    norm_R = N*[0.0]
    vector_R = N*[0,0]
    versor_R = N*[0,0]
    versor_T = N*[0,0]
    
    proj_radial = N*[0]
    radial = N*[0,0]
    
    proj_tangencial= N*[0]
    norm_tangencial= N*[0]
    tangencial= N*[0,0]
    
    angles = N*[0]
    angular= N*[0,0]
    for i in range(N):
        vector_R[i],norm_R[i] = relative_pos(point_cm, agents[i])
        
        versor_R[i] = normalized(vector_R[i])
        proj_radial[i] = np.dot(speeds[i], versor_R[i])
        radial[i] = proj_radial[i]*versor_R[i]
        
        versor_T[i] = versor_R[i]*rot_matrix(90,'deg')
        norm_tangencial[i]= norm_R[i]*np.linalg.norm(angular[i])
        proj_tangencial[i] = np.dot(speeds[i], versor_T[i])
        tangencial[i] = proj_tangencial[i]*versor_T[i]
        
        angular[i] = np.cross(np.array(vector_R[i]),np.array(speeds[i]))
        angles[i] = (np.pi - angle(vector_R[i],speeds[i]))*360/(2*np.pi)
        
    #print proj_radial[0], radial[0]
    #print proj_tangencial[0], norm_tangencial[0], tangencial[0]
    #print angular[0]
    #print angles[0]
    
    
    min_R = min(norm_R)
    max_R = max(norm_R)
    
    #return mean_R, min_R, max_R, 
    #return angles



In [ ]:

##MILL MODEL
def mill (dt, agents, speeds, N, width, height, cr, ca, lr, la, alpha, beta, mass):
    clr = cr / lr
    cla = ca / la
    for i in range(N):
        u_dir = np.array([0.0, 0.0])
        propulsion = np.array([0.0, 0.0])
        friction = np.array([0.0, 0.0])
        grad_U = np.array([0.0, 0.0])
        
        #virtuals = virtualizer(agents[i], agents, height, width, N)
        
        for j in range(N):   #Duality interactions, by the Morse potential  
            if i == j:
                #Eliminate the i-i interaction
                continue

            rel_pos, distance = relative_pos(agents[i], agents[j])
            u_dir = normalized(rel_pos)
            grad_U = grad_U + u_dir * (clr*np.exp(-distance / lr) - cla *np.exp(- distance / la))
            
        
        propulsion = alpha * speeds[i] # self-propulsion propto alpha
        norm = (np.linalg.norm(speeds[i]))
        friction =  beta *((norm)**2) * speeds[i] #friction force propto beta
     
        d_speed = (propulsion - friction - grad_U)/mass
        speeds[i]= speeds[i]+dt*d_speed
        #print u_dir
    
   # print speeds[N-1], propulsion, friction, grad_U
    return

def run(N_steps, N, s, dt, width, height):#N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dev_bias, dTheta, sight_theta):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """
    
    agents, speeds = initialize_agents(s, N, width, height)
    window = initialize_window(agents, width, height)
    old_cm=np.array([0.0,0.0])
    bias = np.array([0.0,-1.0])
    for i in range(N_steps):
        
        #print 'step', i
        
        next_step(agents, speeds, dt, N)
        start = timeit.default_timer()
        ## MODEL
        #couple_speeds(agents, speeds, a, s, N)
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        #couzin(agents, speeds, N, width, height, s, noise, dTheta, rr, ro, ra, sight_theta, 1, 200, 1.5, 1) #(...)last 4 parameters: model2, roa, orient, atract
        
        mill (dt, agents, speeds, N, width, height, cr=120, ca=250, lr=120, la=250.0, alpha=1.0, beta=0.01, mass=1) 
        # Region II, really nice ring
        #mill (dt, agents, speeds, N, width, height, cr=110, ca=100, lr=20, la=100.0, alpha=4.0, beta=0.06, mass=3) 
        # Region VII,(l<1/sqrt(C)) alpha ~ beta, best vortex in this region, decreasing in size with increasing N
        
        mill_observables (N, agents, speeds)
        print i
        #print angles
        
        
        #interaction(agents, speeds)
        #biaser(speeds, N, s, i, prop, bias, dev_bias, weight)
        
        ## BOUNDARY CONDITIONS
        rigid_boundary(width, height, agents, speeds, N)
        #periodic_boundary(width, height, agents, speeds, N)

        #time.sleep(0.02)
        stop = timeit.default_timer()
        #print stop - start
    window.close()
    return

In [ ]:
run(100000, 50, 5, 0.3, 900, 700)

3.60946069139 [ 2.46147913  2.63994829]
[ 0.74436725 -3.01494326] 0.0 [[  3.10829302e-17   2.20512010e+00]
 [  5.07622774e-01  -1.35024664e-16]]
-615.87865188
147.827545393
0
2.38963804029 [ 1.63589424  1.74190126]
[-0.16197194 -2.728132  ] 0.0 [[ -6.78959298e-18   1.98864283e+00]
 [ -1.10882468e-01  -1.21769254e-16]]
-788.123124592
129.585121274
1
0.894090183014 [ 0.61496342  0.64901252]
[-1.30550582 -2.39766753] 0.0 [[ -5.49828974e-17   1.74044663e+00]
 [ -8.97938857e-01  -1.06571620e-16]]
-1014.11276481
103.573661835
2
-0.99451921032 [-0.6879326  -0.71820414]
[-2.76048179 -1.97048779] 0.0 [[ -1.16922599e-16   1.42301173e+00]
 [ -1.90949094e+00  -8.71343379e-17]]
-1299.08515859
78.1284412064
3
-3.34570418524 [-2.32992167 -2.4010834 ]
[-4.53962621 -1.37327393] 0.0 [[ -1.93577435e-16   9.85545958e-01]
 [ -3.16135943e+00  -6.03472852e-17]]
-1624.70298874
60.4974584834
4
-6.01239189225 [-4.21985222 -4.28272151]
[-6.44697385 -0.53853752] 0.0 [[ -2.77068227e-16   3.83608761e-01]
 [ -4.5248

-13.2250572686 [-10.17348619  -8.44998927]
[-6.61121037  6.42365871] 0.0 [[ -3.11411087e-16  -4.10431850e+00]
 [ -5.08572901e+00   2.51317025e-16]]
-22.4018324915
0.81248782528
48
-13.2156930835 [-10.17174771  -8.43742213]
[-6.59899427  6.42183628] 0.0 [[ -3.11002760e-16  -4.09995475e+00]
 [ -5.07906051e+00   2.51049823e-16]]
-20.4575593314
0.768010957839
49
-13.2034847184 [-10.16800706  -8.42280483]
[-6.58532965  6.41852297] 0.0 [[ -3.10531491e-16  -4.09452258e+00]
 [ -5.07136410e+00   2.50717198e-16]]
-18.5991933705
0.723809700946
50
-13.1882500415 [-10.16147915  -8.40679966]
[-6.56984456  6.41411892] 0.0 [[ -3.09960040e-16  -4.08865563e+00]
 [ -5.06203160e+00   2.50357952e-16]]
-16.7440073784
0.676511818075
51
-13.1697922081 [-10.15135466  -8.39007898]
[-6.55223834  6.40898091] 0.0 [[ -3.09254210e-16  -4.08296920e+00]
 [ -5.05050453e+00   2.50009758e-16]]
-14.833335346
0.623223377915
52
-13.1478693585 [-10.1369984   -8.37303601]
[-6.53234835  6.4032466 ] 0.0 [[ -3.08392772e-16  -4.0

-4.22250605795 [ 2.49279439  3.40815691]
[ 1.39547349  8.3815065 ] 0.0 [[ -5.04450580e-17   6.76505585e+00]
 [ -8.23830316e-01  -4.14240200e-16]]
367.096088523
66.6413317102
96
-5.26878799182 [ 2.8675505   4.42009966]
[ 0.4319433   9.14770186] 0.0 [[ -1.43948766e-17   7.67420401e+00]
 [ -2.35086176e-01  -4.69909469e-16]]
350.913574579
61.1892836109
97
-6.13951925562 [ 3.04739009  5.32983211]
[-0.36202374  9.6305032 ] 0.0 [[  1.10030120e-17   8.36042091e+00]
 [  1.79692823e-01  -5.11928136e-16]]
327.600485841
56.4791717311
98
-6.8384047025 [ 3.05697739  6.11708003]
[-0.95442517  9.86219587] 0.0 [[  2.61252328e-17   8.82191740e+00]
 [  4.26657430e-01  -5.40186645e-16]]
300.55321488
52.486918404
99
-7.37570678458 [ 2.92237427  6.77205871]
[-1.34170784  9.88693236] 0.0 [[  3.25515000e-17   9.07775869e+00]
 [  5.31606338e-01  -5.55852406e-16]]
272.291638259
49.2012800925
100
-7.76294863055 [ 2.66626788  7.29070552]
[-1.5349924   9.74994068] 0.0 [[  3.22822766e-17   9.15682297e+00]
 [  5.272

-2.89140655074 [ 2.84468285 -0.51769808]
[ 10.16964914  -0.18938627] 0.0 [[ -6.12648705e-16   3.39090713e-02]
 [ -1.00053126e+01  -2.07633178e-18]]
186.995588495
73.8430386965
144
-2.02516445995 [ 2.02511637 -0.01395676]
[ 10.08340181  -0.01347815] 0.0 [[ -6.17415625e-16   9.28869258e-05]
 [ -1.00831623e+01  -5.68768382e-21]]
184.398238359
78.6289168122
145
-1.17690532055 [ 1.16076165  0.19426406]
[ 9.61878097  0.46637905] 0.0 [[ -5.80901391e-16   7.69821386e-02]
 [ -9.48683966e+00  -4.71379648e-18]]
183.498754528
83.3438748869
146
-0.409153265108 [ 0.38664359  0.13383993]
[ 8.83207407  1.19994078] 0.0 [[ -5.11055839e-16   3.92517923e-01]
 [ -8.34617524e+00  -2.40347909e-17]]
184.141528423
87.6645000673
147
0.233176226733 [-0.20554463 -0.11010248]
[ 7.81450997  2.11734434] 0.0 [[ -4.21797966e-16   9.99779706e-01]
 [ -6.88848353e+00  -6.12188509e-17]]
186.129339607
91.3449210391
148
0.728771663543 [-0.58471999 -0.43498353]
[ 6.66511367  3.14641782] 0.0 [[ -3.27449912e-16   1.87800928e+0

-0.852485282606 [ 0.72365892 -0.4506095 ]
[ 7.6331253   2.42878768] 0.0 [[ -3.96762191e-16  -1.28381664e+00]
 [ -6.47961830e+00   7.86110972e-17]]
201.716411063
85.1572389617
192
-0.236530717967 [ 0.21823591 -0.09121331]
[ 8.59987132  1.40343768] 0.0 [[ -4.85860369e-16  -5.41207476e-01]
 [ -7.93470199e+00   3.31394002e-17]]
197.824416604
88.6454795078
193
0.386337165592 [-0.37590937  0.08915466]
[ 9.28639538  0.61398673] 0.0 [[ -5.53279634e-16  -1.41689138e-01]
 [ -9.03574213e+00   8.67595746e-18]]
195.477932146
92.2346878577
194
0.970266847324 [-0.96779825  0.06916862]
[ 9.6400921   0.11858638] 0.0 [[ -5.88783567e-16  -8.45381555e-03]
 [ -9.61556535e+00   5.17646908e-19]]
194.784721426
95.6780319294
195
1.47427289271 [-1.46890992 -0.12563524]
[ 9.6487467  -0.05551038] 0.0 [[ -5.88666124e-16  -4.73050802e-03]
 [ -9.61364738e+00   2.89660075e-19]]
195.797797986
98.736770829
196
1.86849240337 [-1.81733231 -0.43424318]
[ 9.3376764   0.08666605] 0.0 [[ -5.56112538e-16   2.01414472e-02]
 [ 

...

-0.608571297891 [-0.07881553 -0.60344605]
[ 0.08942399  9.90161129] 0.0 [[  7.09145355e-19  -9.81822220e+00]
 [  1.15812225e-02   6.01192720e-16]]
242.255644746
86.5143212788
14937
-0.580365354217 [-0.00325788 -0.58035621]
[ -2.94359142e-03   9.97514098e+00] 0.0 [[ -1.01179386e-21  -9.97498381e+00]
 [ -1.65238476e-05   6.10791600e-16]]
241.768041954
86.6692377191
14938
-0.560296957424 [ 0.06619668 -0.55637279]
[ 0.20467337  9.74920585] 0.0 [[ -1.48067641e-18  -9.68092504e+00]
 [ -2.41812809e-02   5.92785693e-16]]
241.384253286
86.778260122
14939
-0.551146413259 [ 0.13218547 -0.53506016]
[ 0.70011754  9.24026742] 0.0 [[ -1.02817857e-17  -8.97057263e+00]
 [ -1.67914304e-01   5.49289153e-16]]
241.182574244
86.8264746308
14940
-0.553061045762 [ 0.19780075 -0.5164798 ]
[ 1.45539452  8.47863456] 0.0 [[ -3.18725237e-17  -7.91783026e+00]
 [ -5.20517813e-01   4.84827274e-16]]
241.186649138
86.8134394497
14941
-0.564017178752 [ 0.26501478 -0.49787804]
[ 2.42747236  7.50800931] 0.0 [[ -6.98413957

-0.690275665815 [-0.39757707 -0.56428093]
[ 2.99182342  7.00646981] 0.0 [[  1.05515335e-16  -5.72759185e+00]
 [  1.72319620e+00   3.50713852e-16]]
243.377220207
86.0506035114
14984
-0.686289395116 [-0.3229729  -0.60554244]
[ 1.93207635  8.07846906] 0.0 [[  5.56754748e-17  -7.12797819e+00]
 [  9.09249505e-01   4.36462784e-16]]
243.361373853
86.0781302548
14985
-0.668788704408 [-0.23923303 -0.62453654]
[ 1.05802205  8.95647126] 0.0 [[  2.31743618e-17  -8.36384277e+00]
 [  3.78466050e-01   5.12137664e-16]]
243.165063626
86.1793417183
14986
-0.641339431665 [-0.15334089 -0.62273818]
[ 0.42326809  9.58545275] 0.0 [[  6.19678476e-18  -9.30743860e+00]
 [  1.01201175e-01   5.69916244e-16]]
242.797520847
86.3336100824
14987
-0.609768497829 [-0.071184   -0.60559926]
[ 0.06551565  9.92950081] 0.0 [[  4.68320640e-19  -9.86160866e+00]
 [  7.64825647e-03   6.03849374e-16]]
242.33226004
86.5088687877
14988
-0.580307816128 [ 0.00424033 -0.58029232]
[  4.77288435e-03   9.97168213e+00] 0.0 [[ -2.13551563

-0.573465791039 [-0.53744407 -0.2000422 ]
[ 8.53928877  1.39648704] 0.0 [[  4.90036374e-16  -4.87136892e-01]
 [  8.00290132e+00   2.98285318e-17]]
242.342327668
86.6967092079
15030
-0.5967257198 [-0.52963091 -0.27490487]
[ 7.58302761  2.35269513] 0.0 [[  4.12118449e-16  -1.08386035e+00]
 [  6.73040503e+00   6.63673055e-17]]
242.408628226
86.563023457
15031
-0.625516086148 [-0.51582113 -0.35383462]
[ 6.470747    3.47386768] 0.0 [[  3.26735196e-16  -1.96505683e+00]
 [  5.33599069e+00   1.20325028e-16]]
242.606998168
86.4008380783
15032
-0.65391716284 [-0.48992823 -0.43310275]
[ 5.26655486  4.69376964] 0.0 [[  2.41611273e-16  -3.10877988e+00]
 [  3.94581154e+00   1.90357867e-16]]
242.873569812
86.2437968208
15033
-0.675950208489 [-0.44767951 -0.50645013]
[ 4.04175259  5.93728504] 0.0 [[  1.63909111e-16  -4.44846197e+00]
 [  2.67683893e+00   2.72389736e-16]]
243.122160641
86.1248745129
15034
-0.686897661121 [-0.38811281 -0.56674231]
[ 2.87130941  7.12563501] 0.0 [[  9.93406034e-17  -5.8791

-0.600232737624 [-0.59570273  0.07360433]
[ 9.92762212  0.0773988 ] 0.0 [[  6.03303723e-16   9.49113037e-03]
 [  9.85269750e+00  -5.81164121e-19]]
243.53222714
86.5667606125
15077
-0.580230550064 [ -5.80230425e-01  -3.80324612e-04]
[  9.98694187e+00   3.84615515e-04] 0.0 [[  6.11523689e-16  -2.52104524e-07]
 [  9.98693973e+00   1.54369499e-23]]
243.148460013
86.6750426741
15078
-0.566977559019 [-0.56262673 -0.070105  ]
[ 9.74438819  0.22193768] 0.0 [[  5.92093001e-16  -2.74418992e-02]
 [  9.66961252e+00   1.68033170e-18]]
242.731342903
86.743991288
15079
-0.564803428692 [-0.54766929 -0.13806252]
[ 9.22053729  0.72834605] 0.0 [[  5.47467257e-16  -1.78039451e-01]
 [  8.94081882e+00   1.09017722e-17]]
242.423863786
86.7507755884
15080
-0.574712445296 [-0.53589084 -0.2076425 ]
[ 8.44738095  1.49092609] 0.0 [[  4.82312664e-16  -5.38668716e-01]
 [  7.87676356e+00   3.29839460e-17]]
242.272185731
86.6903854335
15081
-0.595029184431 [-0.52438928 -0.28120387]
[ 7.46947597  2.46703923] 0.0 [[  4

-0.628268055944 [-0.51712622  0.35679297]
[ 7.05404579  2.92450154] 0.0 [[  3.55525461e-16   1.66082230e+00]
 [  5.80617140e+00  -1.01696036e-16]]
243.169274958
86.3973058326
15124
-0.641682592824 [-0.56896312  0.29671118]
[ 8.12286896  1.87443374] 0.0 [[  4.41015810e-16   8.66729820e-01]
 [  7.20233475e+00  -5.30718950e-17]]
243.589685778
86.3274754267
15125
-0.643270842966 [-0.60266337  0.22493165]
[ 8.997707    1.01329425] 0.0 [[  5.16171033e-16   3.54317230e-01]
 [  8.42971268e+00  -2.16956731e-17]]
243.846191986
86.3234342817
15126
-0.63307354729 [-0.61588383  0.14652382]
[ 9.62196328  0.39399278] 0.0 [[  5.73177564e-16   9.11889747e-02]
 [  9.36070000e+00  -5.58371430e-18]]
243.883080155
86.3833453283
15127
-0.614327764356 [-0.61062496  0.06734805]
[ 9.95699677  0.05336732] 0.0 [[  6.06015359e-16   5.85059807e-03]
 [  9.89698189e+00  -3.58245810e-19]]
243.6896847
86.4882096555
15128
-0.593149178297 [-0.59309279 -0.00817875]
[ 9.98368715  0.01007806] 0.0 [[  6.11266408e-16  -1.389

KeyboardInterrupt: 